### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install gym
!pip install pyarrow

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/baselines
Obtaining file:///project/baselines
     |████████████████████████████████| 1.6 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 60.5 MB 26.3 MB/s eta 0:00:01     |█████▉                          | 11.1 MB 22.5 MB/s eta 0:00:03     |██████████▏                     | 19.3 MB 22.5 MB/s eta 0:00:02     |████████████▋                   | 23.8 MB 23.9 MB/s eta 0:00:02     |█████████████▉                  | 26.2 MB 23.9 MB/s eta 0:00:02     |███████████████▊                | 29.7 MB 23.9 MB/s eta 0:00:02     |████████████████▉               | 31.9 MB 23.9 MB/s eta 0:00:02     |█████████████████▋              | 33.4 MB 23.9 MB/s eta 0:00:02     |██████████████████▏             | 34.3 MB 23.9 MB/s eta 0:00:02     |████████████████████████████▉   | 54.6 MB 26.3 MB/s eta 0:00:01     |█████████████████████████████▋  | 55.9 MB 26.3 MB/s eta 0:00:01     |█████████████████████████████

In [25]:
import os
import gym
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import A2C
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session

In [3]:
cicddos2019 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ddos2019/clean/cicddos2019.feather")
cicdos2017 = pd.read_feather("/project/datasets/clean-ids-collection/cic-dos2017/clean/cicdos2017.feather")
cicids2017 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ids2017/clean/cicids2017.feather")
data = pd.concat([cicddos2019], ignore_index=True)
print(len(data.columns))
data.head(5)


69


,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,2018-11-03 09:18:16.964447,114456999,45,0,0.0,0.0,0.0,0.0,0.000000,...,0,8185.583496,28337.111328,98168.0,3.0,9529897.0,351582.62500,10001143.0,9048097.0,Benign
1,0,2018-11-03 09:18:18.506537,114347504,56,0,0.0,0.0,0.0,0.0,0.000000,...,0,35028.417969,121314.914062,420255.0,4.0,9493930.0,351541.09375,9978130.0,8820294.0,Benign
2,6,2018-11-03 09:18:18.610576,36435473,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62416.000000,0.000000,62416.0,62416.0,36373056.0,0.00000,36373056.0,36373056.0,Benign
3,6,2018-11-03 09:18:18.610579,36434705,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62413.000000,0.000000,62413.0,62413.0,36372292.0,0.00000,36372292.0,36372292.0,Benign
4,6,2018-11-03 09:18:18.610581,36434626,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62409.000000,0.000000,62409.0,62409.0,36372216.0,0.00000,36372216.0,36372216.0,Benign


In [4]:
cicdos2017_features = ["Init Bwd Win Bytes",  "Idle Min", "ACK Flag Count", "Fwd Packet Length Min", "Fwd PSH Flags"]
cicids2017_features = ["Protocol", "Avg Bwd Segment Size", "Packet Length Max", "Bwd Packet Length Min", "Fwd IAT Mean"]
cicddos2019_features = ["URG Flag Count", "Down/Up Ratio", "Bwd Packet Length Min", "ACK Flag Count", "Fwd Packets Length Total"]
important_features = cicddos2019_features + ["Label", "Timestamp"] # Adding Timestamp for custom environment logic
important_features = list(set(important_features))
print(len(important_features))

7


In [5]:
removable_features = data.columns ^ important_features
print(removable_features.shape)
skinny_data = data.drop(labels=removable_features, axis='columns')

(62,)


In [6]:
skinny_data.info()
skinny_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138007 entries, 0 to 138006
Data columns (total 7 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Timestamp                 138007 non-null  datetime64[ns]
 1   Fwd Packets Length Total  138007 non-null  float32       
 2   Bwd Packet Length Min     138007 non-null  float32       
 3   ACK Flag Count            138007 non-null  int8          
 4   URG Flag Count            138007 non-null  int8          
 5   Down/Up Ratio             138007 non-null  float32       
 6   Label                     138007 non-null  category      
dtypes: category(1), datetime64[ns](1), float32(3), int8(2)
memory usage: 3.0 MB


,Fwd Packets Length Total,Bwd Packet Length Min,ACK Flag Count,URG Flag Count,Down/Up Ratio
count,1.380070e+05,138007.000000,138007.000000,138007.000000,138007.000000
mean,4.018388e+03,25.685892,0.198584,0.275283,0.702986
std,5.148026e+04,52.287216,0.398936,0.446658,0.937449
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,1.800000e+01,0.000000,0.000000,0.000000,0.000000
50%,8.600000e+01,0.000000,0.000000,0.000000,1.000000
75%,2.064000e+03,46.000000,0.000000,1.000000,1.000000
max,1.526642e+07,1460.000000,1.000000,1.000000,23.000000


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [7]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [8]:
skinny_data.drop('Timestamp', inplace=True, axis=1)
print(skinny_data['Label'].value_counts())
skinny_data['Label'] = skinny_data['Label'].astype('object')
atk_idx = skinny_data.loc[skinny_data['Label'] != "Benign"].index
skinny_data.loc[atk_idx, 'Label'] = 1.0
skinny_data.loc[skinny_data.index.difference(atk_idx), 'Label'] = 0.0
skinny_data['Label'] = skinny_data['Label'].astype(dtype=np.float32)

Benign           95095
DrDoS_NTP        13687
TFTP             13027
Syn               6713
UDP               2158
MSSQL             1763
DrDoS_UDP         1174
UDP-lag           1097
DrDoS_MSSQL        927
DrDoS_DNS          552
LDAP               484
DrDoS_SSDP         413
DrDoS_SNMP         392
Portmap            215
DrDoS_LDAP         174
NetBIOS             73
DrDoS_NetBIOS       51
UDPLag              10
WebDDoS              2
Name: Label, dtype: int64


In [9]:
x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(skinny_data, train_frac=0.9, val_frac=0.0, test_frac=0.1)

Label
Index(['Fwd Packets Length Total', 'Bwd Packet Length Min', 'ACK Flag Count',
       'URG Flag Count', 'Down/Up Ratio'],
      dtype='object')
0.0    95095
1.0    42912
Name: Label, dtype: int64
42912
95095
52183
Int64Index([ 47237,  95217,  60158,  54475,  13008,  43015,  66025,  85204,
             20216, 124992,
            ...
            128219,  69215, 123046,  95868,  67047, 135626,  45187,  32149,
             46261,   1698],
           dtype='int64', length=52183)
1.0    42912
0.0    42912
Name: Label, dtype: int64
38620


In [10]:
print("Training")
print(x_train.shape)
print(y_train.shape)
print("Validation")
print(x_val.shape)
print(y_val.shape)
print("Testing")
print(x_test.shape)
print(y_test.shape)


Training
(77240, 5)
(77240,)
Validation
(0, 5)
(0,)
Testing
(8582, 5)
(8582,)


In [11]:
print(x_train.shape)

(77240, 5)


In [12]:
print(x_train.index)
print(y_train.index)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

Int64Index([131096,  42732,  20162, 133524, 132433,   3134,  31592,  97017,
             40175, 114604,
            ...
             18944, 105278, 120558,  26600, 113002,  81646, 119999,  85039,
            111123, 101908],
           dtype='int64', length=77240)
Int64Index([131096,  42732,  20162, 133524, 132433,   3134,  31592,  97017,
             40175, 114604,
            ...
             18944, 105278, 120558,  26600, 113002,  81646, 119999,  85039,
            111123, 101908],
           dtype='int64', length=77240)


In [13]:
# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


In [14]:
print(x_train.index)
print(y_train.index)

RangeIndex(start=0, stop=77240, step=1)
RangeIndex(start=0, stop=77240, step=1)


In [15]:
print(y_train)

       Label
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
...      ...
77235    1.0
77236    1.0
77237    1.0
77238    1.0
77239    1.0

[77240 rows x 1 columns]


In [16]:
print(y_train.iloc[17162,:])

Label    0.0
Name: 17162, dtype: float32


In [17]:
print(y_train.index)
print(x_train.index)

RangeIndex(start=0, stop=77240, step=1)
RangeIndex(start=0, stop=77240, step=1)


### Custom environment

In [18]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
        # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(important_features) - 2,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [27]:
def ids_dqn():
#     logger.configure(dir=os.path.join(os.getcwd(),'Training/Logs/ids_dqn'), format_strs=['stdout', 'tensorboard'])
    env = IdsEnv(images_per_episode=4096)
    env = bench.Monitor(env, logger.get_dir())

    model = A2C(
        MlpPolicy, 
        env, 
        verbose=1, 
    ).learn(
        total_timesteps=int(1.0e5),
        log_interval=1000
    )

    model.save('ddqn_stableb2_ids.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = ids_dqn()
print("DQN Training Time:", time.time() - start_time)


Wrapping the env in a DummyVecEnv.


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------
| explained_variance | 0.302    |
| fps                | 11       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 3.53     |
---------------------------------
---------------------------------
| ep_len_mean        | 4.1e+03  |
| ep_reward_mean     | 2.12e+03 |
| explained_variance | -4.14    |
| fps                | 268      |
| nupdates           | 1000     |
| policy_entropy     | 0.44     |
| total_timesteps    | 5000     |
| value_loss         | 0.438    |
---------------------------------
---------------------------------
| ep_len_mean        | 4.1e+03  |
| ep_reward_mean     | 2.5e+03  |
| explained_variance | -0.106   |
| fps                | 274      |
| nupdates           | 2000     |
| policy_entropy     | 0.0192   |
| total_timesteps    | 10000    |
| value_loss         | 0.0968   |
---------------------------------
---------------------------------
| ep_len_mean 

In [28]:
def ids_dqn_eval(dqn_model):
    attempts, correct = 0,0

    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model.predict(obs)[0])

                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

ids_dqn_eval(dqn_model)

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 86.03729603729604%


##### 